Adrian Wiegman, 2023-08-07

A hydrologically enforced groundwater elevation model using water table elevations, lakes, flowlines, and cranberry bogs. From the hydroenforced elevation model I calculate flow direction, then flow accumulation. 

In [36]:
# iphython options
# delete variables in workspace
%reset -f
#places plots inline
%matplotlib inline
#automatically reloads modules if they are changed
%load_ext autoreload 
%autoreload 2
# this codeblock sets up the environment from jupyter notebooks
setup_notebook = "C:/Users/Adrian.Wiegman/Documents/GitHub/Wiegman_USDA_ARS/MEP/_Setup.ipynb"
%run $setup_notebook # magic command to run the notebook

# Create new file geodatabase to store results
gdb = "gwbogsheds.gdb"
# only run this once
#if not os.path.exists(os.path.join(wdr,gdb)):
#    arcpy.management.CreateFileGDB(wdr,gdb)
ap.env.workspace = os.path.join(wdr,gdb)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
***
loading python modules...

  `module_list` contains names of all loaded modules

...module loading complete

***
loading user defined functions...

type `fn_`+TAB to for autocomplete suggestions

 the object `def_list` contains user defined function names:
   fn_get_info
   fn_arcgis_table_to_df
   fn_arcgis_table_to_np_to_pd_df
   fn_run_script_w_propy_bat
   fn_try_mkdir
   fn_hello
   fn_recursive_glob_search
   fn_regex_search_replace
   fn_regex_search_0
   fn_arcpy_table_to_excel
   fn_agg_sum_df_on_group
   fn_add_prefix_suffix_to_selected_cols
   fn_calc_pct_cover_within_groups
   fn_buildWhereClauseFromList

 use ??{insert fn name} to inspect
 for example running `??fn_get_info` returns:
Signature: fn_get_info(name='fn_get_info')
Source:   
def fn_get_info(name='fn_get_info'):
    '''
    returns the source information about a given function name
    '''
    get_ipython().run_line_magic

## Create burn raster

In [37]:
bogs_raw = arcpy.management.CopyFeatures(
    in_features=os.path.join(wdr,"outputs","WMAbogsDRAFT2013_copy"),
    out_feature_class="bogs_Raw")
# these bogs have been split into pieces rather than permit ids. 10/23/2023
bogs = arcpy.management.CopyFeatures(
    in_features=r"C:\Workspace\Geodata\MEP\Default.gdb\bogs_buff_neg2m_pos5m_neg5m_split",
    out_feature_class="bogs_split")
flowlines = arcpy.management.CopyFeatures(
    in_features=os.path.join(r"C:\Workspace\Geodata\Massachusetts\NHDPLUS_H_0109_HU4_GDB\NHDPLUS_H_0109_HU4_GDB.gdb","NHDPlusBurnLineEvent"),
    out_feature_class="flowlines_Raw")
waterbodies = arcpy.management.CopyFeatures(
    in_features=os.path.join(r"C:\Workspace\Geodata\Massachusetts\NHDPLUS_H_0109_HU4_GDB\NHDPLUS_H_0109_HU4_GDB.gdb","NHDPlusBurnWaterbody"),
    out_feature_class="waterbodies_Raw")

In [38]:
'''
# commented out 10/23/2023 
bogs_buffered = arcpy.analysis.Buffer(
    in_features="bogs_Raw",
    out_feature_class="bogs_buf10m",
    buffer_distance_or_field="10 Meters",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="ALL",
    dissolve_field=None,
    method="PLANAR"
)
'''
# added 10/23/2023
bogs_buffered = arcpy.analysis.Buffer(
    in_features="bogs_split",
    out_feature_class="bogs_buf10m",
    buffer_distance_or_field="10 Meters",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="ALL",
    dissolve_field=None,
    method="PLANAR"
)
flowlines_buffered = arcpy.analysis.Buffer(
    in_features="flowlines_Raw",
    out_feature_class="flowlines_buf10m",
    buffer_distance_or_field="10 Meters",
    line_side="FULL",
    line_end_type="ROUND",
    dissolve_option="ALL",
    dissolve_field=None,
    method="PLANAR"
)

In [39]:
arcpy.management.Dissolve(
    in_features="waterbodies_Raw",
    out_feature_class="waterbodies_diss",
    dissolve_field=None,
    statistics_fields=None,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

<Result 'C:\\Workspace\\Geodata\\MEP\\gwbogsheds.gdb\\waterbodies_diss'>

In [40]:
# why did I choose the depths shown here? 

arcpy.management.CalculateField(
    in_table="flowlines_buf10m",
    field="burn",
    expression="2",
    expression_type="PYTHON3",
    code_block="",
    field_type="SHORT",
    enforce_domains="NO_ENFORCE_DOMAINS")

arcpy.management.CalculateField(
    in_table="waterbodies_diss",
    field="burn",
    expression="2",
    expression_type="PYTHON3",
    code_block="",
    field_type="SHORT",
    enforce_domains="NO_ENFORCE_DOMAINS")

arcpy.management.CalculateField(
    in_table="bogs_buf10m",
    field="burn",
    expression="0.5",
    expression_type="PYTHON3",
    code_block="",
    field_type="SHORT",
    enforce_domains="NO_ENFORCE_DOMAINS")

arcpy.analysis.Union(
    in_features="bogs_buf10m #;flowlines_buf10m #;waterbodies_diss #",
    out_feature_class=r"C:\Workspace\Geodata\MEP\gwbogsheds.gdb\bogs_flowlines_waterbodies_Union",
    join_attributes="ALL",
    cluster_tolerance=None,
    gaps="GAPS"
)

arcpy.management.Dissolve(
    in_features="bogs_flowlines_waterbodies_Union",
    out_feature_class=r"C:\Workspace\Geodata\MEP\gwbogsheds.gdb\bogs_flowlines_wate_Dissolve",
    dissolve_field="OBJECTID",
    statistics_fields=None,
    multi_part="MULTI_PART",
    unsplit_lines="DISSOLVE_LINES",
    concatenation_separator=""
)

<Result 'C:\\Workspace\\Geodata\\MEP\\gwbogsheds.gdb\\bogs_flowlines_wate_Dissolve'>

In [41]:
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_19N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-69.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', snapRaster="gw_elev_meters", extent='346823.303957774 4597731.43403507 422133.303957774 4662251.43403507 PROJCS["NAD_1983_UTM_Zone_19N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-69.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', cellSize="gw_elev_meters"):
    arcpy.conversion.FeatureToRaster(
        in_features="waterbodies_diss",
        field="burn",
        out_raster="waterbody_burn_rast"
    )
    arcpy.conversion.FeatureToRaster(
        in_features="flowlines_buf10m",
        field="burn",
        out_raster="flowline_burn_rast"
    )
    arcpy.conversion.FeatureToRaster(
        in_features="bogs_buf10m",
        field="burn",
        out_raster="bog_burn_rast"
    )
    arcpy.conversion.FeatureToRaster(
        in_features="bogs_flowlines_waterbodies_Union",
        field="burn",
        out_raster="combined_burn_rast"
    )

In [42]:
burn = arcpy.ia.Con(
    in_conditional_raster="combined_burn_rast",
    in_true_raster_or_constant=0,
    in_false_raster_or_constant=1,
    where_clause="Value IS NULL")
burn.save("combined_burn_rast_null")

In [43]:
burn = arcpy.ia.Con(
    in_conditional_raster="bog_burn_rast",
    in_true_raster_or_constant=0,
    in_false_raster_or_constant=1,
    where_clause="Value IS NULL")
burn.save("bog_burn_rast_null")

In [44]:
burn = arcpy.ia.Con(
    in_conditional_raster="flowline_burn_rast",
    in_true_raster_or_constant=0,
    in_false_raster_or_constant=1,
    where_clause="Value IS NULL")
burn.save("flowline_burn_rast_null")

In [45]:
burn = arcpy.ia.Con(
    in_conditional_raster="waterbody_burn_rast",
    in_true_raster_or_constant=0,
    in_false_raster_or_constant=1,
    where_clause="Value IS NULL")
burn.save("waterbody_burn_rast_null")

## Create hydroenforced (Burned and Filled) elevation model

In [48]:
# load masked gw elevation raster
gw_elev_meters_raw = arcpy.sa.Raster(os.path.join(wdr,"gwdepth.gdb","gw_elev_meters_masked"))
gw_elev_meters_raw.save("gw_elev_meters_raw")

In [50]:
gw_elev_meters = arcpy.sa.ExtractByMask(
    in_raster=gw_elev_meters_raw,
    in_mask_data=r"C:\Workspace\Geodata\MEP\Default.gdb\NHDPlusLand",
    extraction_area="INSIDE",
    analysis_extent='346823.303957774 4597731.43403507 422133.303957774 4662251.43403507 PROJCS["NAD_1983_UTM_Zone_19N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-69.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'
)
gw_elev_meters.save("gw_elev_meters")

In [51]:
burn_bog = arcpy.sa.Raster("bog_burn_rast_null")
burn_waterbody = arcpy.sa.Raster("waterbody_burn_rast_null")
burn_flowline = arcpy.sa.Raster("flowline_burn_rast_null")
burn_combined = arcpy.sa.Raster("combined_burn_rast_null")

In [54]:
# burn in lakes, streams, and bogs
gw_elev_meters_b = gw_elev_meters - burn_combined
#gw_elev_meters_b = gw_elev_meters - burn_waterbody - burn_bog - burn_flowline
gw_elev_meters_b.save("gwe_b")

In [55]:
out_surface_raster = arcpy.sa.Fill(
    in_surface_raster="gwe_b",
    z_limit=None
)
out_surface_raster.save("gwe_bf")

burn in lakes, streams, and bogs assuming that gw only flows to areas where gw depth is less than 1 m

In [63]:
# burn in lakes, streams, and bogs assuming that gw only flows to areas where gw depth is less than 1 m
burn_logic = arcpy.sa.Raster(os.path.join(wdr,"gwdepth.gdb","gw_depth_le_1m"))
gw_elev_meters_b = gw_elev_meters - burn_bog/2*burn_logic - burn_flowline - burn_waterbody
gw_elev_meters_b.save("gwe_b_lt1m")

In [64]:
out_surface_raster = arcpy.sa.Fill(
    in_surface_raster="gwe_b_lt1m",
    z_limit=None
)
out_surface_raster.save("gwe_bf_lt1m")

In [65]:
out_surface_raster = arcpy.sa.Fill(
    in_surface_raster="gwe_b_lt1m_noflow",
    z_limit=None
)
out_surface_raster.save("gwe_bf_lt1m_noflow")

In [66]:
# groundwater elevation data

# raw groundwater elevation
# this assumes moderate flow to cranberry bogs as flow
gwe = os.path.join(wdr,"gwdepth.gdb","gw_elev_meters")

# burned with 1m for all waterbodies, streams, bogs
# since sinks are not filled
# this assumes a high amount of gw flow bypassing cranberry bogs
gweb = os.path.join(wdr,gdb,"gwe_b")

# burned and filled for all waterbodies streams and bogs
# this assumes the highest amount of flow to cranberry bogs
gwebf = os.path.join(wdr,gdb,"gwe_bf")

# burned only for waterbodies streams and bogs where gw depth is less than 1m
# this assumes moderate flow bypassing bogs as sinks are not filled
gweblt1m = os.path.join(wdr,gdb,"gwe_b_lt1m")

# burned and filled for waterbodies streams and bogs where gw depth is less than 1m
# this assumes flow bypasses bogs where gw is greater than 1m deep
gwebflt1m = os.path.join(wdr,gdb,"gwe_bf_lt1m")

# burned only for waterbodies streams and bogs where gw depth is less than 1m
# this assumes moderate flow bypassing bogs as sinks are not filled
gweblt1m = os.path.join(wdr,gdb,"gwe_b_lt1m_noflow")

elev_rasters = [gwe,gweb,gwebf,gweblt1m,gwebflt1m]
suffixes = ["gwe","gwe_b","gwe_bf","gwe_b_lt1m","gwe_bf_lt1m","gwe_b_lt1m_noflow"]

In [67]:
# calculate flow direction
i=0
for er in elev_rasters:
    print("calculating D8 flow direction for {}".format(er))
    D8 = arcpy.sa.FlowDirection(in_surface_raster=er,
                                force_flow="FORCE",
                                flow_direction_type="D8")
    _ = D8
    _.save("D8_{}".format(suffixes[i]))
    i+=1
    #if i == 2:
    #   break

calculating D8 flow direction for C:\Workspace\Geodata\MEP\gwdepth.gdb\gw_elev_meters
calculating D8 flow direction for C:\Workspace\Geodata\MEP\gwbogsheds.gdb\gwe_b
calculating D8 flow direction for C:\Workspace\Geodata\MEP\gwbogsheds.gdb\gwe_bf
calculating D8 flow direction for C:\Workspace\Geodata\MEP\gwbogsheds.gdb\gwe_b_lt1m_noflow
calculating D8 flow direction for C:\Workspace\Geodata\MEP\gwbogsheds.gdb\gwe_bf_lt1m


In [68]:
# calculate flow accumulation for d8
i=0
for suffix in suffixes:
    print("calculating D8 flow accumulation for {}".format(suffix))
    FA = arcpy.sa.FlowAccumulation(
        in_flow_direction_raster=os.path.join(wdr,gdb,"D8_{}".format(suffix)),
        in_weight_raster=None,
        data_type="FLOAT",
        flow_direction_type="D8")
    _ = FA
    FA.save(os.path.join(wdr,gdb,"FA_D8_{}".format(suffix)))
    i+=1
#    if i == 2:
#        break

calculating D8 flow accumulation for gwe
calculating D8 flow accumulation for gwe_b
calculating D8 flow accumulation for gwe_bf
calculating D8 flow accumulation for gwe_b_lt1m
calculating D8 flow accumulation for gwe_bf_lt1m
calculating D8 flow accumulation for gwe_b_lt1m_noflow


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000865: Input flow direction raster: C:\Workspace\Geodata\MEP\gwbogsheds.gdb\D8_gwe_b_lt1m_noflow does not exist.
Failed to execute (FlowAccumulation).


In [69]:
# calculate flow Dinf direction
i=0
for er in elev_rasters:
    print("calculating D inf flow direction for {}".format(er))
    Dinf = arcpy.sa.FlowDirection(in_surface_raster=er,
                                force_flow="FORCE",
                                flow_direction_type="DINF")
    _ = Dinf
    _.save("Dinf_{}".format(suffixes[i]))
    i+=1
    #if i == 2:
    #   break

calculating D inf flow direction for C:\Workspace\Geodata\MEP\gwdepth.gdb\gw_elev_meters
calculating D inf flow direction for C:\Workspace\Geodata\MEP\gwbogsheds.gdb\gwe_b
calculating D inf flow direction for C:\Workspace\Geodata\MEP\gwbogsheds.gdb\gwe_bf
calculating D inf flow direction for C:\Workspace\Geodata\MEP\gwbogsheds.gdb\gwe_b_lt1m_noflow
calculating D inf flow direction for C:\Workspace\Geodata\MEP\gwbogsheds.gdb\gwe_bf_lt1m


In [70]:
# calculate flow accumulation for Dinf
i=0
for suffix in suffixes:
    print("calculating D inf flow accumulation for {}".format(suffix))
    FA = arcpy.sa.FlowAccumulation(
        in_flow_direction_raster=os.path.join(wdr,gdb,"Dinf_{}".format(suffix)),
        in_weight_raster=None,
        data_type="FLOAT",
        flow_direction_type="DINF")
    _ = FA
    FA.save(os.path.join(wdr,gdb,"FA_Dinf_{}".format(suffix)))
    i+=1
#    if i == 2:
#        break

calculating D inf flow accumulation for gwe
calculating D inf flow accumulation for gwe_b
calculating D inf flow accumulation for gwe_bf
calculating D inf flow accumulation for gwe_b_lt1m
calculating D inf flow accumulation for gwe_bf_lt1m
calculating D inf flow accumulation for gwe_b_lt1m_noflow


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000865: Input flow direction raster: C:\Workspace\Geodata\MEP\gwbogsheds.gdb\Dinf_gwe_b_lt1m_noflow does not exist.
Failed to execute (FlowAccumulation).


## Generate Bog Pour Points

In [64]:
# clip bogs to mask layer
arcpy.analysis.Clip(
    in_features="bogs_Raw",
    clip_features=os.path.join(wdr,"gwdepth.gdb","contour_mask_poly"),
    out_feature_class="bogs_clip",
    cluster_tolerance=None
)

<Result 'C:\\Workspace\\Geodata\\MEP\\gwbogsheds.gdb\\bogs_clip'>

In [65]:
selection = arcpy.management.SelectLayerByLocation(
    in_layer="bogs_Raw",
    overlap_type="COMPLETELY_WITHIN",
    select_features=os.path.join(wdr,"gwdepth.gdb","contour_mask_poly"),
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

arcpy.management.CopyFeatures(
    in_features=selection,
    out_feature_class="bogs",)

<Result 'C:\\Workspace\\Geodata\\MEP\\gwbogsheds.gdb\\bogs'>

In [101]:
# select flow accumulation data
suffixes = ["gwe","gwe_bf","gwe_bf_lt1m","gwe_bf_lt1m_noflow"]
elev_source = "gwe_bf_lt1m_noflow"
eid = elev_source

In [102]:
# Zonal statistics to find the maximum flow accumulation inside each bog
method = "D8"

out_raster = arcpy.sa.ZonalStatistics(
    in_zone_data="bogs_clip",
    zone_field="OBJECTID",
    in_value_raster="FA_{}_{}".format(method,eid),
    statistics_type="MAXIMUM",
    ignore_nodata="DATA",
    process_as_multidimensional="CURRENT_SLICE",
    percentile_value=90,
    percentile_interpolation_type="AUTO_DETECT",
    circular_calculation="ARITHMETIC",
    circular_wrap_value=360
)
out_raster.save(r"bog_Max_FA_{}_{}".format(method,eid))

In [103]:
# Find points that equal of maximum value of flow accumulation inside each bog
outEqualTo = arcpy.ia.EqualTo("bog_Max_FA_{}_{}".format(method,eid),"FA_{}_{}".format(method,eid))
outEqualTo.save("bog_Max_FA_{}_EQ_{}".format(method,eid))

In [104]:
# Set 0 as null value 
out_raster = arcpy.sa.SetNull(
    in_conditional_raster="bog_Max_FA_{}_EQ_{}".format(method,eid),
    in_false_raster_or_constant="bog_Max_FA_{}_EQ_{}".format(method,eid),
    where_clause="Value = 0"
)
out_raster.save("bog_Max_FA_{}_EQ_{}_NULL".format(method,eid))

In [105]:
# raster to point
arcpy.conversion.RasterToPoint(
    in_raster="bog_Max_FA_{}_EQ_{}_NULL".format(method,eid),
    out_point_features="bogPourPoints_{}_{}".format(method,eid),
    raster_field="Value"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\gwbogsheds.gdb\\bogPourPoints_D8_gwe_bf_lt1m_noflow'>

In [84]:
# Store area as variable
inFeature = "bogs"
fieldName = "bogsurf_m2"
# Create a new field
arcpy.management.AddField(inFeature, fieldName, "DOUBLE")
# Calculate field
arcpy.management.CalculateField(inFeature, fieldName, 
                                "0",
                                "PYTHON3")

arcpy.management.CalculateGeometryAttributes(
    in_features=inFeature,
    geometry_property="bogsurf_m2 AREA_GEODESIC",
    length_unit="",
    area_unit="SQUARE_METERS",
    coordinate_system='PROJCS["NAD_1983_StatePlane_Massachusetts_Mainland_FIPS_2001",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Lambert_Conformal_Conic"],PARAMETER["False_Easting",200000.0],PARAMETER["False_Northing",750000.0],PARAMETER["Central_Meridian",-71.5],PARAMETER["Standard_Parallel_1",41.71666666666667],PARAMETER["Standard_Parallel_2",42.68333333333333],PARAMETER["Latitude_Of_Origin",41.0],UNIT["Meter",1.0]]',
    coordinate_format="SAME_AS_INPUT")

<Result 'bogs'>

In [108]:
# Identity
arcpy.analysis.Identity(
    in_features="bogPourPoints_{}_{}".format(method,eid),
    identity_features="bogs",
    out_feature_class="bogPourPoints_Ident_{}_{}".format(method,eid),
    join_attributes="ALL",
    cluster_tolerance=None,
    relationship="NO_RELATIONSHIPS"
)

<Result 'C:\\Workspace\\Geodata\\MEP\\gwbogsheds.gdb\\bogPourPoints_Ident_D8_gwe_bf_lt1m_noflow'>

## Delineate Bog Watersheds

In [109]:
method="D8"
eid="gwe_bf_lt1m_noflow"
outdir = os.path.join(odr,"bogsheds",eid)
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [110]:
# Make a numpy array with the bog FID from Bog Pour Point Identity
inLyr = "bogPourPoints_Ident_{}_{}".format(method,eid)
idCol = "FID_bogs"
_ = arcpy.da.FeatureClassToNumPyArray(inLyr,idCol)
# convert to data frame 
df_pourpoint = pd.DataFrame(_)
print(df_pourpoint.iloc[0:4,:])

   FID_bogs
0       612
1       396
2       119
3       394


In [111]:
# select dataframe
df = df_pourpoint#.iloc[0:4,:]
# create list to store output filenames
outfiles = [None]*len(df)
print(len(df))

637


In [112]:
%%time 
# this cell took a Wall time: 4h 7min 26s for 982 watersheds
# loop through all rows of the cranberry bog pour points contained dataframe 
for i in range(len(df)):
    print("{} out of {}".format(i,len(df)))
    bogFID = df.iloc[i,0]

    print("{} = {}".format(idCol,bogFID))
    
    # clear the selection
    arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=inLyr,
        selection_type="CLEAR_SELECTION")
    
    # Select pour point by Attributes
    selection = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=inLyr,
        selection_type="NEW_SELECTION",
        where_clause="{} = {}".format(idCol,bogFID))
                      
    # Delineate Watershed
    out_raster = arcpy.sa.Watershed(
        in_flow_direction_raster="D8_"+eid,
        in_pour_point_data=selection,
        pour_point_field=idCol)
    
    # Save Watershed to outfile with name = bogFID
    tmpfile2 = os.path.join(outdir,"bogWS{}.tif".format(bogFID))
    
    # create and save temporary output file
    out_raster.save(tmpfile2)
    
    outfile = os.path.join(outdir,"bogWS_poly{}.shp".format(bogFID))
    arcpy.conversion.RasterToPolygon(
        in_raster=tmpfile2,
        out_polygon_features=outfile,
        simplify="SIMPLIFY",
        create_multipart_features="MULTIPLE_OUTER_PART",
        raster_field="Value")
    
    print(outfile)
    # Assign the name to the ith value of outfiles list
    outfiles[i] = outfile

0 out of 637
FID_bogs = 612
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly612.shp
1 out of 637
FID_bogs = 396
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly396.shp
2 out of 637
FID_bogs = 119
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly119.shp
3 out of 637
FID_bogs = 394
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly394.shp
4 out of 637
FID_bogs = 114
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly114.shp
5 out of 637
FID_bogs = 395
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly395.shp
6 out of 637
FID_bogs = 118
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly118.shp
7 out of 637
FID_bogs = 172
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly172.shp
8 out of 637
FID_bogs = 117
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly117.shp
9 out of 637
FID_bogs = 419
C:\Worksp

C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly448.shp
152 out of 637
FID_bogs = 614
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly614.shp
153 out of 637
FID_bogs = 363
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly363.shp
154 out of 637
FID_bogs = 301
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly301.shp
155 out of 637
FID_bogs = 413
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly413.shp
156 out of 637
FID_bogs = 408
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly408.shp
157 out of 637
FID_bogs = 427
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly427.shp
158 out of 637
FID_bogs = 426
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly426.shp
159 out of 637
FID_bogs = 24
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly24.shp
160 out of 637
FID_bogs = 438
C:\Workspace\Geodata\

C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly491.shp
302 out of 637
FID_bogs = 279
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly279.shp
303 out of 637
FID_bogs = 608
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly608.shp
304 out of 637
FID_bogs = 529
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly529.shp
305 out of 637
FID_bogs = 305
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly305.shp
306 out of 637
FID_bogs = 299
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly299.shp
307 out of 637
FID_bogs = 311
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly311.shp
308 out of 637
FID_bogs = 53
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly53.shp
309 out of 637
FID_bogs = 493
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly493.shp
310 out of 637
FID_bogs = 509
C:\Workspace\Geodata\

C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly597.shp
452 out of 637
FID_bogs = 256
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly256.shp
453 out of 637
FID_bogs = 308
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly308.shp
454 out of 637
FID_bogs = 445
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly445.shp
455 out of 637
FID_bogs = -1
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly-1.shp
456 out of 637
FID_bogs = 598
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly598.shp
457 out of 637
FID_bogs = 325
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly325.shp
458 out of 637
FID_bogs = 326
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly326.shp
459 out of 637
FID_bogs = 375
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly375.shp
460 out of 637
FID_bogs = 173
C:\Workspace\Geodata\

C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly320.shp
602 out of 637
FID_bogs = 482
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly482.shp
603 out of 637
FID_bogs = 225
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly225.shp
604 out of 637
FID_bogs = 226
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly226.shp
605 out of 637
FID_bogs = 71
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly71.shp
606 out of 637
FID_bogs = 315
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly315.shp
607 out of 637
FID_bogs = 224
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly224.shp
608 out of 637
FID_bogs = 7
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly7.shp
609 out of 637
FID_bogs = 316
C:\Workspace\Geodata\MEP\outputs\bogsheds\gwe_bf_lt1m_noflow\bogWS_poly316.shp
610 out of 637
FID_bogs = 386
C:\Workspace\Geodata\MEP\

In [113]:
outdir = os.path.join(odr,"bogsheds",eid)
import glob
import os
outfiles = glob.glob(os.path.join(outdir,"*.shp"))

merge = arcpy.management.Merge(
    inputs=outfiles,
    output="bogsheds_{}".format(eid))

In [114]:
# for some reason extra features are added to the output that have a polygon area ~= the extent
# the line below removes those features. 
select = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=merge,
    selection_type="NEW_SELECTION",
    where_clause="gridcode = 0",
    invert_where_clause=None)

arcpy.management.DeleteRows(in_rows=select)

arcpy.management.CopyFeatures(
    in_features=select,
    out_feature_class=r"bogsheds_select_{}".format(eid))

<Result 'C:\\Workspace\\Geodata\\MEP\\gwbogsheds.gdb\\bogsheds_select_gwe_bf_lt1m_noflow'>